In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
species = pd.read_csv("../MNE/Test_Patri.csv", sep = ";")
species

,RNFO\n70 mailles,Milieu,TAXREF v16_TXT,VERNACULAIRE,Présence 53,Indigénat 53,LRR_53_manuel,Reg PDL,LRN1,LR PdL,ZNIEFF,LRMA,STATUT,X_L93,Y L_93
0,Apium inundatum (L.) Rchb.f.,Aquatique,"Helosciadium inundatum (L.) W.D.J.Koch, 1824",Ache inondée,P,I,EN,NaN,NaN,NT,X,Anx 1,4,431259,6804260
1,Callitriche brutia Petagna,Aquatique,"Callitriche brutia Petagna, 1787",Callitriche pédonculé,P,I,NT,NaN,NaN,NaN,NaN,NaN,1,431259,6804260
2,Callitriche obtusangula Le Gall,Aquatique,"Callitriche obtusangula Le Gall, 1852",Callitriche à angles obtus,P,I,VU,NaN,NaN,NaN,NaN,NaN,1,431259,6804260
3,Callitriche palustris L.,Aquatique,"Callitriche palustris L., 1753",Callitriche des marais,P,I,NT,NaN,NaN,VU,X,NaN,3,431259,6804260
4,Callitriche truncata Guss. subsp. occidentalis...,Aquatique,Callitriche truncata subsp. occidentalis (Rouy...,Callitriche de l'Ouest,P,I,CR*,NaN,NaN,NT,X,NaN,3,431259,6804260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Verbascum virgatum Stokes,NaN,"Verbascum virgatum Stokes, 1787",Molène en baguette,P,I,VU,NaN,NaN,NaN,NaN,NaN,1,431259,6804260
408,Veronica acinifolia L.,NaN,"Veronica acinifolia L., 1762",Véronique à feuilles d'acinos,P,I,VU,NaN,NaN,NaN,NaN,NaN,1,431259,6804260
409,Veronica agrestis L.,NaN,"Veronica agrestis L., 1753",Véronique agreste,P,I,VU,NaN,NaN,NaN,NaN,NaN,1,431259,6804260
410,Vicia lathyroides L.,NaN,"Vicia lathyroides L., 1753",Vesce fausse gesse,P,I,VU,NaN,NaN,NaN,NaN,NaN,1,431259,6804260


In [4]:
#patrimonial_species_df = species.dropna(subset=['LRR_53_manuel'])
#patrimonial_species_df

In [ ]:
unique_patrimonial_species = species['VERNACULAIRE'].unique()

# Générer des couleurs aléatoires
np.random.seed(42)  # Pour la reproductibilité
colors = ['#' + ''.join([np.random.choice(list('0123456789ABCDEF')) for _ in range(6)]) for _ in unique_patrimonial_species]

# Créer le mapping espèce-couleur
patrimonial_species_color_mapping = dict(zip(unique_patrimonial_species, colors))

In [12]:
unique_patrimonial_species = species['TAXREF v16_TXT'].unique()

# Générer des couleurs aléatoires
np.random.seed(42)  # Pour la reproductibilité
colors = ['#' + ''.join([np.random.choice(list('0123456789ABCDEF')) for _ in range(6)]) for _ in unique_patrimonial_species]

# Créer le mapping espèce-couleur
patrimonial_species_color_mapping_latin = dict(zip(unique_patrimonial_species, colors))

In [6]:
#flore = gpd.read_file("../MNE/BDD_Bota_Habitat.gpkg", layer="BDD_Flore_ponctuelle", driver="GPKG")

In [ ]:
import json

with open('patrimonial_species_color_mapping.json', 'w') as file:
    json.dump(patrimonial_species_color_mapping, file)

In [14]:
with open('patrimonial_species_color_mapping_latin.json', 'w') as file:
    json.dump(patrimonial_species_color_mapping_latin, file)

In [ ]:
import os

# Changez le répertoire de travail
os.chdir('C:/Users/lored/Documents/MNE')

# Chargez les mappages des espèces
with open('patrimonial_species_color_mapping.json', 'r') as file:
    patrimonial_species_color_mapping = json.load(file)

from qgis.core import QgsProject, QgsCategorizedSymbolRenderer, QgsRendererCategory, QgsMarkerSymbol
from PyQt5.QtGui import QColor
import random

# Liste des formes disponibles selon la documentation de QGIS 3.28
shapes = [
    QgsSimpleMarkerSymbolLayer.Circle,
    QgsSimpleMarkerSymbolLayer.Square,
    QgsSimpleMarkerSymbolLayer.Diamond,
    QgsSimpleMarkerSymbolLayer.Pentagon,
    QgsSimpleMarkerSymbolLayer.Hexagon,
    QgsSimpleMarkerSymbolLayer.Triangle,
    QgsSimpleMarkerSymbolLayer.Star,
    QgsSimpleMarkerSymbolLayer.Cross,
    # Ajoutez d'autres formes ici selon les besoins
]

# Obtenir la couche sur laquelle appliquer le rendu
layer = QgsProject.instance().mapLayersByName("BDD_Flore_ponctuelle")[0]

# Configuration du rendu catégorisé
categories = []
for species, color in patrimonial_species_color_mapping.items():
    symbol = QgsMarkerSymbol()
    simpleMarker = symbol.symbolLayer(0)
    
    # Définir la couleur
    simpleMarker.setColor(QColor(color))
    
    # Assigner une forme aléatoire
    shape = random.choice(shapes)
    simpleMarker.setShape(shape)
    
    category = QgsRendererCategory(species, symbol, species)
    categories.append(category)

renderer = QgsCategorizedSymbolRenderer("Cat_VERNACULAIRE", categories)
layer.setRenderer(renderer)

# Rafraîchissement de la couche pour appliquer le nouveau rendu
layer.triggerRepaint()

In [ ]:
# Changez le répertoire de travail
os.chdir('C:/Users/lored/Documents/MNE')

# Chargez les mappages des espèces
with open('patrimonial_species_color_mapping_latin.json', 'r') as file:
    patrimonial_species_color_mapping_latin = json.load(file)

from qgis.core import QgsProject, QgsCategorizedSymbolRenderer, QgsRendererCategory, QgsMarkerSymbol
from PyQt5.QtGui import QColor
import random

# Liste des formes disponibles selon la documentation de QGIS 3.28
shapes = [
    QgsSimpleMarkerSymbolLayer.Circle,
    QgsSimpleMarkerSymbolLayer.Square,
    QgsSimpleMarkerSymbolLayer.Diamond,
    QgsSimpleMarkerSymbolLayer.Pentagon,
    QgsSimpleMarkerSymbolLayer.Hexagon,
    QgsSimpleMarkerSymbolLayer.Triangle,
    QgsSimpleMarkerSymbolLayer.Star,
    QgsSimpleMarkerSymbolLayer.Cross,
    # Ajoutez d'autres formes ici selon les besoins
]

# Obtenir la couche sur laquelle appliquer le rendu
layer = QgsProject.instance().mapLayersByName("BDD_Flore_ponctuelle")[0]

# Configuration du rendu catégorisé
categories = []
for species, color in patrimonial_species_color_mapping_latin.items():
    symbol = QgsMarkerSymbol()
    simpleMarker = symbol.symbolLayer(0)
    
    # Définir la couleur
    simpleMarker.setColor(QColor(color))
    
    # Assigner une forme aléatoire
    shape = random.choice(shapes)
    simpleMarker.setShape(shape)
    
    category = QgsRendererCategory(species, symbol, species)
    categories.append(category)

renderer = QgsCategorizedSymbolRenderer("Cat_TAXREF v16_FORMULE", categories)
layer.setRenderer(renderer)

# Rafraîchissement de la couche pour appliquer le nouveau rendu
layer.triggerRepaint()